Chloe Chen
Updated: 2/15/2023


### Import packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from helper import *

### Connect to MySQL Database

In [6]:
from connection import *

con = connect()

### TODO: why is the error popping up? doesn't affect function though

2023-02-21 15:43:07,805| ERROR   | Password is required for key /home/chloe/.ssh/id_rsa


In [3]:
# test
query = "show tables;"
SQL(query, con)

,Tables_in_constant_therapy
0,action_images
1,actions
2,anti_distractor_categories
3,anti_distractor_categories_v2
4,assessment_tasks
...,...
124,word_spelling_tasks
125,word_task_content
126,words
127,words_to_features


### Basic plots

In [13]:
query = "select * from domain_scores;"
data = SQL(query, con)

In [20]:
domain_groups = data.groupby("domain_id")
domain_groups["baseline_domain_score"].mean().sort_values()

domain_id
5     0.121016
12    0.257091
8     0.269387
9     0.281605
4     0.288451
7     0.333839
2     0.397512
10    0.412056
13    0.415907
1     0.419652
6     0.538331
3     0.559260
14    0.610745
11    0.843765
Name: baseline_domain_score, dtype: float64

In [22]:
query = "select * from before_after_v2;"
data = SQL(query, con)

In [26]:
data["acc_diff"] = data["after_accuracy_avg"] - data["before_accuracy_avg"]
data["acc_diff"].mean()

0.01632128142303705

In [28]:
data["lat_diff"] = data["after_log_latency_avg"] - data["before_log_latency_avg"]
data["lat_diff"].mean()

-0.13203832363196075

In [25]:
data.head()

,patient_id,session_types,disorder_id,task_type_id,task_level,before_accuracy_avg,after_accuracy_avg,before_latency_avg,after_latency_avg,before_log_latency_avg,after_log_latency_avg,timestamp,completed_task_count,acc_diff
0,2043,SCHEDULED,2,1,1,0.555556,0.777778,19.127039,15.477067,2.885246,2.706310,2016-11-09 07:54:08,20,0.222222
1,2043,SCHEDULED,2,9,1,0.888889,0.777778,11.483053,11.466350,2.419956,2.434754,2016-11-09 08:50:01,20,-0.111111
2,2043,SCHEDULED,2,16,1,1.000000,1.000000,9.127086,7.867066,2.145456,2.055330,2016-11-09 10:00:03,20,0.000000
3,2043,SCHEDULED,2,30,1,0.944444,1.000000,12.883181,10.707060,2.495407,2.326532,2016-11-09 12:15:03,20,0.055556
4,2043,SCHEDULED,3,1,1,0.555556,0.777778,19.127039,15.477067,2.885246,2.706310,2016-11-15 20:12:00,20,0.222222


In [ ]:
# borrowing Claire's table for now, takes around 3.5 mins
query = "select * from q3;"
data = SQL(query, con)

In [ ]:
data.head()

In [ ]:
## current accuracy

sns.boxplot(data=data, x="accuracy")

In [ ]:
## current task levels

tl_data = data.groupby("task_level")
tl_data.size().plot.pie()